###### **1º Passo: Local salvar os dados**

In [1]:
from pyspark.sql.types import *
from pyspark.sql import functions as F

StatementMeta(, 34a46ae2-9431-4525-91d4-5fb0edb0ccdd, 3, Finished, Available, Finished)

In [2]:
raw_dados_publicos_abfss_files = "abfss://3ed9004c-a27b-47fb-8a19-3a0a8508fbf3@onelake.dfs.fabric.microsoft.com/34b49a7f-cf79-410e-8c7a-2b00f7d44771/Files"

StatementMeta(, 34a46ae2-9431-4525-91d4-5fb0edb0ccdd, 4, Finished, Available, Finished)

###### **2º Passo: Definição do schema dos dados**

In [3]:
df_import = spark.read.parquet("Files/Mdic_Fatos/Importacao/IMP_2025.parquet")
df_export = spark.read.parquet("Files/Mdic_Fatos/Exportacao/EXP_2025.parquet")

StatementMeta(, 34a46ae2-9431-4525-91d4-5fb0edb0ccdd, 5, Finished, Available, Finished)

In [4]:
schema_import = df_import.schema
schema_export = df_export.schema

StatementMeta(, 34a46ae2-9431-4525-91d4-5fb0edb0ccdd, 6, Finished, Available, Finished)

In [5]:
schema_import = StructType([
    StructField(
        field.name,
        field.dataType,
        field.nullable
    )
    for field in schema_import
])

schema_export = StructType([
    StructField(
        field.name,
        field.dataType,
        field.nullable
    )
    for field in schema_export
])

StatementMeta(, 34a46ae2-9431-4525-91d4-5fb0edb0ccdd, 7, Finished, Available, Finished)

###### **3º Passo: Spark Structured Streaming**
**https://learn.microsoft.com/en-us/fabric/data-engineering/lakehouse-streaming-data**

In [6]:
path_monitore = f"{raw_dados_publicos_abfss_files}/{'Mdic_Fatos'}"


StatementMeta(, 34a46ae2-9431-4525-91d4-5fb0edb0ccdd, 8, Finished, Available, Finished)

In [13]:
tb_ft_import = spark.readStream \
    .schema(schema_import) \
    .format("parquet") \
    .load(f"{path_monitore}/{'Importacao'}") \
    .withColumn("file_path", F.col("_metadata.file_path")) \
    .withColumn("file_modification_time", F.col("_metadata.file_modification_time")) \
    .withColumn("write_timestamp", F.current_timestamp()) 

query = tb_ft_import.writeStream \
    .outputMode("append") \
    .format("delta") \
    .option("checkpointLocation", "Files/Mdic_Fatos/Importacao/checkpoint_import") \
    .trigger(availableNow=True) \
    .toTable("tb_ft_import")

#########################################

tb_ft_export = spark.readStream \
    .schema(schema_export) \
    .format("parquet") \
    .load(f"{path_monitore}/{'Exportacao'}") \
    .withColumn("file_path", F.col("_metadata.file_path")) \
    .withColumn("file_modification_time", F.col("_metadata.file_modification_time")) \
    .withColumn("write_timestamp", F.current_timestamp())

query = tb_ft_export.writeStream \
    .outputMode("append") \
    .format("delta") \
    .option("checkpointLocation", "Files/Mdic_Fatos/Exportacao/checkpoint_export") \
    .trigger(availableNow=True) \
    .toTable("tb_ft_export")

query.awaitTermination()


StatementMeta(, 34a46ae2-9431-4525-91d4-5fb0edb0ccdd, 9, Finished, Available, Finished)